https://colab.research.google.com/drive/1wOlV3fKvPztK2YPSY93Rm6HAvik9DYUU

In [0]:
import numpy as np
import gym
import random

In [0]:
gym.Env

gym.core.Env

In [0]:
from gym import envs
print(envs.registry.all())

dict_values([EnvSpec(Copy-v0), EnvSpec(RepeatCopy-v0), EnvSpec(ReversedAddition-v0), EnvSpec(ReversedAddition3-v0), EnvSpec(DuplicatedInput-v0), EnvSpec(Reverse-v0), EnvSpec(CartPole-v0), EnvSpec(CartPole-v1), EnvSpec(MountainCar-v0), EnvSpec(MountainCarContinuous-v0), EnvSpec(Pendulum-v0), EnvSpec(Acrobot-v1), EnvSpec(LunarLander-v2), EnvSpec(LunarLanderContinuous-v2), EnvSpec(BipedalWalker-v2), EnvSpec(BipedalWalkerHardcore-v2), EnvSpec(CarRacing-v0), EnvSpec(Blackjack-v0), EnvSpec(KellyCoinflip-v0), EnvSpec(KellyCoinflipGeneralized-v0), EnvSpec(FrozenLake-v0), EnvSpec(FrozenLake8x8-v0), EnvSpec(CliffWalking-v0), EnvSpec(NChain-v0), EnvSpec(Roulette-v0), EnvSpec(Taxi-v2), EnvSpec(GuessingGame-v0), EnvSpec(HotterColder-v0), EnvSpec(Reacher-v2), EnvSpec(Pusher-v2), EnvSpec(Thrower-v2), EnvSpec(Striker-v2), EnvSpec(InvertedPendulum-v2), EnvSpec(InvertedDoublePendulum-v2), EnvSpec(HalfCheetah-v2), EnvSpec(Hopper-v2), EnvSpec(Swimmer-v2), EnvSpec(Walker2d-v2), EnvSpec(Ant-v2), EnvSpec(Hum

### Creating the Environment

In [0]:
from gym.envs.registration import register
register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.8196, # optimum = .8196, changing this seems have no influence
)

In [0]:
env = gym.make("FrozenLakeNotSlippery-v0")

In [0]:
env.render()


SFFF
FHFH
FFFH
HFFG


### Create Q Table and Intialize it

In [0]:
action_size=env.action_space.n
state_size=env.observation_space.n
print("State Size {}, Action Size {}".format(state_size,action_size))

State Size 16, Action Size 4


In [0]:
qtable=np.zeros((state_size,action_size))

In [0]:
qtable.shape

(16, 4)

In [0]:
env.

<TimeLimit<FrozenLakeEnv<FrozenLakeNotSlippery-v0>>>

In [0]:
qtable

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [0]:
state=env.reset()

In [0]:
for i in range(100):
    state=env.reset()
    for step in range(20):
        action=env.action_space.sample()
        new_state,reward,done,info=env.step(action)
        qtable[state,action]+=0.9*(reward+0.9*np.max(qtable[new_state,:])-qtable[state,action])
        state=new_state
        total_rewards+=reward
print(qtable)

[[0.         0.         0.34519166 0.        ]
 [0.         0.         0.42616254 0.37971082]
 [0.37657272 0.4782969  0.         0.42616254]
 [0.38742049 0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.59049    0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.729      0.         0.4782969 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.729      0.9        0.        ]
 [0.         0.         0.         0.        ]]


### Hyper Parameters

In [0]:
total_episodes=15000
learning_rate=0.8
max_steps=99
gamma=0.95

epsilon=1
max_epsilon=1
min_epsilon=0.01
decay_rate=0.005

### Q Learning Algorithm

In [0]:
rewards=[]

for episode in range(total_episodes):
    state=env.reset()
    step=0
    done=False
    total_rewards=0
    
    for step in range(max_steps):
        exp_exp_tradeoff=random.uniform(0,1)
        
        ## Exploitation:
        if exp_exp_tradeoff>epsilon:
            action=np.argmax(qtable[state,:])
        else:
            ## Exploration
            action=env.action_space.sample()
            
        new_state,reward,done,info=env.step(action)
        
        ## Update the Q
        qtable[state,action]=qtable[state,action]+learning_rate*(reward+gamma*np.max(qtable[new_state,:])-
                                                                 qtable[state,action])
        
        total_rewards+=reward
        
        state=new_state
        
        if done:
            break
    episode+=1
    epsilon=min_epsilon+(max_epsilon-min_epsilon)*np.exp(decay_rate*episode)
    rewards.append(total_rewards)
print("Score over Time:",sum(rewards)/total_episodes)
print(qtable)           

Score over Time: 0.0148
[[0.73509189 0.77378094 0.77378094 0.73509189]
 [0.73509189 0.         0.81450625 0.77378094]
 [0.77378094 0.857375   0.77378094 0.81450625]
 [0.81450625 0.         0.77378094 0.77378094]
 [0.77378094 0.81450625 0.         0.73509189]
 [0.         0.         0.         0.        ]
 [0.         0.9025     0.         0.81450625]
 [0.         0.         0.         0.        ]
 [0.81450625 0.         0.857375   0.77378094]
 [0.81450625 0.9025     0.9025     0.        ]
 [0.857375   0.95       0.         0.857375  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.9025     0.95       0.857375  ]
 [0.9025     0.95       1.         0.9025    ]
 [0.         0.         0.         0.        ]]


In [0]:
env.step(action)

(12, 0, True, {'prob': 1.0})

## Play Frozen Lake 

In [0]:
env.reset()

for episode in range(1):
    state=env.reset()
    step=0
    done=False
    print("-----------------------")
    print("Episode",episode)
    for step in range(max_steps):
        env.render()
        action=np.argmax(qtable[state,:])
        print(action)
        new_state,reward,done,info=env.step(action)
        
        if done:
            #env.render()
            print("Number of Steps",step+1)
            break
        state=new_state

env.close()
    

-----------------------
Episode 0

SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
2
Number of Steps 6


# Play Frozen Lake Slippery 

In [0]:
env = gym.make("FrozenLake-v0")

In [0]:
env.reset()

for episode in range(5):
    state=env.reset()
    step=0
    done=False
    print("-----------------------")
    print("Episode",episode)
    for step in range(max_steps):
        env.render()
        action=np.argmax(qtable[state,:])
        print(action)
        new_state,reward,done,info=env.step(action)
        
        if done:
            #env.render()
            print("Number of Steps",step+1)
            break
        state=new_state

env.close()
    

-----------------------
Episode 0

SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
2
Number of Steps 4
-----------------------
Episode 1

SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
1
Number of Steps 4
-----------------------
Episode 2

SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
1
Number of Steps 2
-----------------------
Episode 3

SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
1
Number of Steps 3
-----------------------
Episode 4

SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
1
Number of Steps 5


# Frozen Lake with 8 by 8

In [0]:
from gym.envs.registration import register
register(
    id='FrozenLakeNotSlippery-v1',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '8x8', 'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.8196, # optimum = .8196, changing this seems have no influence
)

In [0]:
env = gym.make("FrozenLakeNotSlippery-v1")

In [0]:
env.render()


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [0]:
action_size=env.action_space.n
state_size=env.observation_space.n
print("State Size {}, Action Size {}".format(state_size,action_size))

qtable=np.zeros((state_size,action_size))

State Size 64, Action Size 4


In [0]:
rewards=[]

for episode in range(total_episodes):
    state=env.reset()
    step=0
    done=False
    total_rewards=0
    
    for step in range(max_steps):
        exp_exp_tradeoff=random.uniform(0,1)
        
        ## Exploitation:
        if exp_exp_tradeoff>epsilon:
            action=np.argmax(qtable[state,:])
        else:
            ## Exploration
            action=env.action_space.sample()
            
        new_state,reward,done,info=env.step(action)
        
        ## Update the Q
        qtable[state,action]=qtable[state,action]+learning_rate*(reward+gamma*np.max(qtable[new_state,:])-
                                                                 qtable[state,action])
        
        total_rewards+=reward
        
        state=new_state
        
        if done:
            break
    episode+=1
    epsilon=min_epsilon+(max_epsilon-min_epsilon)*np.exp(decay_rate*episode)
    rewards.append(total_rewards)
print("Score over Time:",sum(rewards)/total_episodes)
print(qtable)           

Score over Time: 0.0010666666666666667
[[0.48767498 0.51334208 0.51334208 0.48767498]
 [0.48767498 0.54036009 0.54036009 0.51334208]
 [0.51334208 0.56880009 0.56880009 0.54036009]
 [0.54036009 0.59873694 0.59873694 0.56880009]
 [0.56880009 0.63024941 0.63024941 0.59873694]
 [0.59873694 0.66342043 0.66342043 0.63024941]
 [0.63024941 0.6983373  0.6983373  0.66342043]
 [0.66342043 0.73509189 0.6983373  0.6983373 ]
 [0.51334208 0.54036006 0.54036009 0.48767498]
 [0.51334208 0.56880006 0.56880009 0.51334208]
 [0.54036009 0.59873691 0.59873694 0.54036009]
 [0.56880009 0.         0.63024941 0.56880009]
 [0.59873694 0.66342043 0.66342043 0.59873694]
 [0.63024941 0.6983373  0.6983373  0.63024941]
 [0.66342043 0.73509189 0.73509189 0.66342043]
 [0.6983373  0.77378094 0.73509189 0.6983373 ]
 [0.54036006 0.56880006 0.56880006 0.51334208]
 [0.54036006 0.59873691 0.59873691 0.54036009]
 [0.56880006 0.63024937 0.         0.56880009]
 [0.         0.         0.         0.        ]
 [0.         0.698337

In [0]:
env.reset()

for episode in range(1):
    state=env.reset()
    step=0
    done=False
    print("-----------------------")
    print("Episode",episode)
    for step in range(max_steps):
        env.render()
        action=np.argmax(qtable[state,:])
        print(action)
        new_state,reward,done,info=env.step(action)
        
        if done:
            #env.render()
            print("Number of Steps",step)
            break
        state=new_state

env.close()

-----------------------
Episode 0

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
2
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
2
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
2
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
1
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
2
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
2
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
2
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
2
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
1
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
1
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
1
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
